In [1]:
import joblib
import pandas as pd
import numpy as np
import sys
import os

def calculate_ckd_stage(df):
    g_stage = np.select(
        [
            df['GFR'] >= 90,
            (df['GFR'] >= 60) & (df['GFR'] < 90),
            (df['GFR'] >= 45) & (df['GFR'] < 60),
            (df['GFR'] >= 30) & (df['GFR'] < 45),
            (df['GFR'] >= 15) & (df['GFR'] < 30),
            df['GFR'] < 15
        ],
        [1, 2, 3, 4, 5, 5],
        default=0
    )
    
    a_stage = np.select(
        [
            df['ACR'] < 30,
            (df['ACR'] >= 30) & (df['ACR'] <= 300),
            df['ACR'] > 300
        ],
        [1, 2, 3],
        default=0
    )
    
    df['ckd_stage'] = g_stage * 10 + a_stage
    
    return df

def calculate_htn_dm(df):
    
    df['htn'] = ((df['SystolicBP'] >= 140) | (df['DiastolicBP'] >= 90)).astype(int)
    df['dm'] = ((df['FastingBloodSugar'] >= 126) | (df['HbA1c'] >= 6.5)).astype(int)
    
    return df


def predict_ckd_risk(patient_data: dict, model_bundle: dict) -> tuple:
    model = model_bundle['model']
    scaler = model_bundle['scaler']
    optimal_threshold = model_bundle['optimal_threshold']
    cols_to_scale = model_bundle['cols_to_scale']
    ckd_features = model_bundle['ckd_features']

    try:
        data_df = pd.DataFrame([patient_data], columns=ckd_features)
    except KeyError as e:
        print(f"Error: Missing or incorrect feature in input data: {e}", file=sys.stderr)
        return None, None
    except Exception as e:
        print(f"An unexpected error occurred during DataFrame creation: {e}", file=sys.stderr)
        return None, None

    data_df = calculate_ckd_stage(data_df)
    data_df = calculate_htn_dm(data_df)
    
    data_scaled = data_df.copy().astype(float)
    
    if cols_to_scale:
        data_scaled.loc[:, cols_to_scale] = scaler.transform(data_df[cols_to_scale])

    y_pred_proba = model.predict_proba(data_scaled)[:, 1]
    risk_score = y_pred_proba[0]
    
    prediction_class = int(risk_score >= optimal_threshold)
    
    return risk_score, prediction_class


if __name__ == '__main__':
    MODEL_FILENAME = 'ckd_stacking_model.joblib'

    if not os.path.exists(MODEL_FILENAME):
        print(f"Error: Model file '{MODEL_FILENAME}' not found.", file=sys.stderr)
        print("Please ensure you run the training script ('Modified_Meta.py') first to create it.", file=sys.stderr)
        sys.exit(1)

    try:
        deployment_bundle = joblib.load(MODEL_FILENAME)
        print(f"Successfully loaded model bundle from '{MODEL_FILENAME}'.")
    except Exception as e:
        print(f"Error loading model bundle: {e}", file=sys.stderr)
        sys.exit(1)
        
   
    # Example 1: Low Risk Patient (Hypothetical Healthy patient features)
    patient_data_low_risk = {
        'age': 75,
        'sc': 4.9, 
        'GFR': 52,  
        'hemo': 15,
        'al': 1, 
        'SystolicBP': 175,
        'DiastolicBP': 100,
        'FastingBloodSugar': 111,
        'HbA1c': 6.4,
        'CholesterolTotal': 230,
        'CholesterolLDL': 99,
        'CholesterolHDL': 72,
        'CholesterolTriglycerides': 138,
        'ACR': 133, 
    }

    # Example 2: High Risk Patient (Hypothetical CKD patient features)
    patient_data_high_risk = {
        'age': 39,
        'sc': 4, 
        'GFR': 79, 
        'hemo': 15, 
        'al': 0.0, 
        'SystolicBP': 122,
        'DiastolicBP': 77,
        'FastingBloodSugar': 70,
        'HbA1c': 8.0,
        'CholesterolTotal': 268,
        'CholesterolLDL': 110,
        'CholesterolHDL': 39,
        'CholesterolTriglycerides': 39,
        'ACR': 5, 
    }


    print("\n" + "="*50)
    print("Prediction for Low Risk Patient (Example 1)")
    print("="*50)
    risk_score, prediction = predict_ckd_risk(patient_data_low_risk, deployment_bundle)
    
    if risk_score is not None:
        print(f"Optimal Threshold: {deployment_bundle['optimal_threshold']:.4f}")
        print(f"Predicted Risk Score: {risk_score:.4f}")
        print(f"Final Prediction Class: {prediction} ({'CKD Detected' if prediction == 1 else 'No CKD'})")
        
    print("\n" + "="*50)
    print("Prediction for high Risk Patient (Example 2)")
    print("="*50)
    risk_score, prediction = predict_ckd_risk(patient_data_high_risk, deployment_bundle)
    
    if risk_score is not None:
        print(f"Optimal Threshold: {deployment_bundle['optimal_threshold']:.4f}")
        print(f"Predicted Risk Score: {risk_score:.4f}")
        print(f"Final Prediction Class: {prediction} ({'CKD Detected' if prediction == 1 else 'No CKD'})")

Successfully loaded model bundle from 'ckd_stacking_model.joblib'.

Prediction for Low Risk Patient (Example 1)
Optimal Threshold (from training): 0.4960
Predicted Risk Score (P(CKD=1)): 0.0002
Final Prediction Class: 0 (No CKD)

Prediction for high Risk Patient (Example 2)
Optimal Threshold (from training): 0.4960
Predicted Risk Score (P(CKD=1)): 0.9896
Final Prediction Class: 1 (CKD Detected)
